In [ ]:
import numpy as np
import pandas as pd
from cvxpy import *
from cvxopt import *
from alphamind.cython.optimizers import QPOptimizer

# Data Preparing
--------------------------

In [ ]:
risk_penlty = 0.5

In [ ]:
sec_cov_values_full = np.genfromtxt('sec_cov_values.csv', delimiter=',')
signal_full = np.genfromtxt('signal.csv', delimiter=',')

In [ ]:
n = 200

sec_cov_values = sec_cov_values_full[:n, :n]
signal = signal_full[:n]

# Optimizing Weights
-------------------------------------

In [ ]:
%%time
w = Variable(n)

lbound = 0.
ubound = 1. / n * 20

objective = Minimize(risk_penlty * quad_form(w, sec_cov_values)  - signal * w)
constraints = [w >= lbound,
               w <= ubound,
               sum_entries(w) == 1,]

prob = Problem(objective, constraints)

In [ ]:
%%time
prob.solve(verbose=True)

In [ ]:
prob.status, prob.value

In [ ]:
%%time
prob.solve(verbose=True, solver='CVXOPT')

In [ ]:
prob.status, prob.value

In [ ]:
%%time
P = matrix(sec_cov_values)
q = -matrix(signal)

G = np.zeros((2*n, n))
h = np.zeros(2*n)
for i in range(n):
    G[i, i] = 1.
    h[i] = 1. / n * 20
    G[i+n, i] = -1.
    h[i+n] = 0.
    
G = matrix(G)
h = matrix(h)
    
A = np.ones((1, n))
b = np.ones(1)

A = matrix(A)
b = matrix(b)

sol = solvers.qp(P, q, G, h, A, b)

In [ ]:
%%time
lbound = np.zeros(n)
ubound = np.ones(n) * 20 / n
qpopt = QPOptimizer(signal, sec_cov_values, lbound, ubound, 1.)
qpopt.feval()
qpopt.status()

# Performace Timing
-------------------------

In [ ]:
import datetime as dt

In [ ]:
def time_function(py_callable, n):
    start = dt.datetime.now()
    py_callable(n)
    return (dt.datetime.now() - start).total_seconds()

In [ ]:
def cvxpy(n):
    w = Variable(n)

    lbound = 0.
    ubound = 0.01

    objective = Minimize(risk_penlty * quad_form(w, sec_cov_values)  - signal * w)
    constraints = [w >= lbound,
                   w <= ubound,
                   sum_entries(w) == 1,]

    prob = Problem(objective, constraints)
    prob.solve(verbose=False, solver='CVXOPT', display=False)

In [ ]:
def cvxopt(n):
    P = matrix(sec_cov_values)
    q = -matrix(signal)

    G = np.zeros((2*n, n))
    h = np.zeros(2*n)
    for i in range(n):
        G[i, i] = 1.
        h[i] = 0.01
        G[i+n, i] = -1.
        h[i+n] = 0.

    G = matrix(G)
    h = matrix(h)

    A = np.ones((1, n))
    b = np.ones(1)

    A = matrix(A)
    b = matrix(b)
    
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)

In [ ]:
def ipopt(n):
    lbound = np.zeros(n)
    ubound = np.ones(n) * 0.01
    qpopt = QPOptimizer(signal, sec_cov_values, lbound, ubound, 1.)
    qpopt.feval()

In [ ]:
n_steps = list(range(200, 3201, 200))
cvxpy_times = [None] * len(n_steps)
cvxopt_times = [None] * len(n_steps)
ipopt_times = [None] * len(n_steps)
print("{0:<8}{1:>12}{2:>12}{3:>12}".format('Scale(n)', 'cvxpy', 'cvxopt', 'ipopt'))

for i, n in enumerate(n_steps):
    sec_cov_values = sec_cov_values_full[:n, :n]
    signal = signal_full[:n]
    cvxpy_times[i] = time_function(cvxpy, n) * 1000
    cvxopt_times[i] = time_function(cvxopt, n) * 1000
    ipopt_times[i] = time_function(ipopt, n) * 1000
    
    print("{0:<8}{1:>12.2f}{2:>12.2f}{3:>12.2f}".format(n, cvxpy_times[i], cvxopt_times[i], ipopt_times[i]))

In [ ]:
df = pd.DataFrame({'cvxpy': cvxpy_times,
                   'cvxopt': cvxopt_times,
                   'ipopt': ipopt_times},
                  index=n_steps)
df.index.name = 'Problem Scale (n)'
df